# 01 - Getting Started: Environment Setup & Validation

This notebook will guide you through setting up and validating your Google Cloud MLOps environment.

## Overview

This is **Phase 1** of our MLOps pipeline project. We'll:

1. **Validate Python Environment** - Check required packages
2. **Configure Google Cloud Authentication** - Set up credentials
3. **Verify API Access** - Ensure required services are enabled
4. **Set Up Cloud Storage** - Create and configure GCS bucket
5. **Test Vertex AI Connectivity** - Verify ML platform access
6. **Prepare Sample Dataset** - Download and process Iris dataset
7. **Environment Summary** - Validate complete setup

---

## 🏗️ Section 1: Python Environment Validation

First, let's verify our Python environment and install required packages.

In [1]:
import sys
import subprocess
from pathlib import Path

print(f"Python version: {sys.version}")
print(f"Python executable: {sys.executable}")
print(f"Current working directory: {Path.cwd()}")

Python version: 3.13.7 (main, Aug 14 2025, 11:12:11) [Clang 17.0.0 (clang-1700.0.13.3)]
Python executable: /Users/farishussain/GCP_MLOps/venv/bin/python
Current working directory: /Users/farishussain/GCP_MLOps/notebooks


In [ ]:
# Install requirements if needed
# Uncomment the line below if you need to install packages
# !pip install -r ../requirements.txt

In [2]:
# Verify key packages
import importlib

required_packages = [
    'google.cloud.aiplatform',
    'google.cloud.storage', 
    'pandas',
    'numpy',
    'sklearn',
    'kfp'
]

print("📦 Package Validation:")
for package in required_packages:
    try:
        mod = importlib.import_module(package)
        version = getattr(mod, '__version__', 'Unknown')
        print(f"  ✅ {package}: {version}")
    except ImportError:
        print(f"  ❌ {package}: Not installed")

📦 Package Validation:
  ✅ google.cloud.aiplatform: 1.128.0
  ✅ google.cloud.storage: 3.6.0
  ✅ pandas: 2.3.3
  ✅ numpy: 2.3.5
  ✅ sklearn: 1.7.2
  ✅ kfp: 2.14.6


## 🔐 Section 2: Google Cloud Authentication

Set up authentication for Google Cloud services.

In [3]:
import os
from google.cloud import aiplatform
from google.auth import default

# Initialize Vertex AI with your project settings
PROJECT_ID = "mlops-295610"  # Replace with your project ID
REGION = "us-central1"

# Set environment variable for project
os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID

print(f"🔧 Project Configuration:")
print(f"  Project ID: {PROJECT_ID}")
print(f"  Region: {REGION}")

🔧 Project Configuration:
  Project ID: mlops-295610
  Region: us-central1


In [4]:
# Test authentication
try:
    credentials, project = default()
    print(f"✅ Authentication successful!")
    print(f"  Authenticated project: {project}")
    
    # Initialize Vertex AI
    aiplatform.init(project=PROJECT_ID, location=REGION)
    print(f"✅ Vertex AI initialized successfully!")
    
except Exception as e:
    print(f"❌ Authentication failed: {e}")
    print("\n🔧 Quick fix: Run the following command in your terminal:")
    print("  gcloud auth application-default login")

✅ Authentication successful!
  Authenticated project: mlops-295610
✅ Vertex AI initialized successfully!


## 🌐 Section 3: Google Cloud APIs Verification

Verify that required Google Cloud APIs are enabled.

In [5]:
from google.cloud import storage
from google.api_core import exceptions

# Test Cloud Storage API
try:
    storage_client = storage.Client(project=PROJECT_ID)
    # List first few buckets to test API access
    buckets = list(storage_client.list_buckets())
    print(f"✅ Cloud Storage API: Working ({len(buckets)} buckets found)")
except exceptions.Forbidden:
    print("❌ Cloud Storage API: Access denied - check IAM permissions")
except Exception as e:
    print(f"❌ Cloud Storage API: {e}")

✅ Cloud Storage API: Working (0 buckets found)


In [6]:
# Test Vertex AI API
try:
    # Try to list models to test Vertex AI API access
    models = aiplatform.Model.list()
    print(f"✅ Vertex AI API: Working ({len(models)} models found)")
except exceptions.Forbidden:
    print("❌ Vertex AI API: Access denied - check IAM permissions")
except Exception as e:
    print(f"✅ Vertex AI API: Working (empty project is normal)")

✅ Vertex AI API: Working (0 models found)


## 🪣 Section 4: Cloud Storage Setup

Create and configure the GCS bucket for our MLOps pipeline.

In [7]:
from google.cloud import storage
from google.api_core import exceptions

BUCKET_NAME = f"mlops-vertex-ai-bucket-{PROJECT_ID.split('-')[-1]}"

storage_client = storage.Client(project=PROJECT_ID)

print(f"🪣 Setting up bucket: {BUCKET_NAME}")

# Check if bucket exists
try:
    bucket = storage_client.bucket(BUCKET_NAME)
    bucket.reload()  # This will raise exception if bucket doesn't exist
    print(f"✅ Bucket already exists: gs://{BUCKET_NAME}")
except exceptions.NotFound:
    # Create bucket
    try:
        bucket = storage_client.create_bucket(
            BUCKET_NAME, 
            location=REGION
        )
        print(f"✅ Created bucket: gs://{BUCKET_NAME}")
    except Exception as e:
        print(f"❌ Failed to create bucket: {e}")
        print("Note: Bucket names must be globally unique")

🪣 Setting up bucket: mlops-vertex-ai-bucket-295610
✅ Created bucket: gs://mlops-vertex-ai-bucket-295610


In [8]:
# Create folder structure in bucket
folders = ['data/', 'models/', 'pipelines/', 'outputs/', 'artifacts/']

for folder in folders:
    blob = bucket.blob(folder + '.gitkeep')
    try:
        blob.upload_from_string('')
        print(f"  📁 Created folder: {folder}")
    except Exception as e:
        print(f"  ❌ Failed to create {folder}: {e}")

print(f"\n✅ Bucket structure created successfully!")
print(f"   Bucket URL: https://console.cloud.google.com/storage/browser/{BUCKET_NAME}")

  📁 Created folder: data/
  📁 Created folder: models/
  📁 Created folder: pipelines/
  📁 Created folder: outputs/
  📁 Created folder: artifacts/

✅ Bucket structure created successfully!
   Bucket URL: https://console.cloud.google.com/storage/browser/mlops-vertex-ai-bucket-295610


## 🤖 Section 5: Vertex AI Connectivity Test

Test our connection to Vertex AI and explore available services.

In [9]:
# Test Vertex AI connection and list available resources
print(f"🤖 Vertex AI Status:")
print(f"  Project: {aiplatform.initializer.global_config.project}")
print(f"  Location: {aiplatform.initializer.global_config.location}")

# List existing resources (will be empty for new projects)
try:
    datasets = aiplatform.TabularDataset.list()
    models = aiplatform.Model.list()
    endpoints = aiplatform.Endpoint.list()
    
    print(f"\n📊 Current Resources:")
    print(f"  Datasets: {len(datasets)}")
    print(f"  Models: {len(models)}")
    print(f"  Endpoints: {len(endpoints)}")
    
    print(f"\n✅ Vertex AI connectivity confirmed!")
    
except Exception as e:
    print(f"❌ Vertex AI connection issue: {e}")

🤖 Vertex AI Status:
  Project: mlops-295610
  Location: us-central1

📊 Current Resources:
  Datasets: 0
  Models: 0
  Endpoints: 0

✅ Vertex AI connectivity confirmed!


## 📊 Section 6: Sample Dataset Preparation

Download and prepare the Iris dataset for our MLOps pipeline.

In [10]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
import pickle

# Load Iris dataset
print("📊 Loading Iris dataset...")
iris = load_iris()
X, y = iris.data, iris.target

# Create DataFrame
df = pd.DataFrame(X, columns=iris.feature_names)
df['target'] = y
df['target_name'] = df['target'].map({0: 'setosa', 1: 'versicolor', 2: 'virginica'})

print(f"\n📈 Dataset Summary:")
print(f"  Shape: {df.shape}")
print(f"  Features: {list(iris.feature_names)}")
print(f"  Classes: {list(iris.target_names)}")
print(f"  \n{df.head()}")

📊 Loading Iris dataset...

📈 Dataset Summary:
  Shape: (150, 6)
  Features: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
  Classes: [np.str_('setosa'), np.str_('versicolor'), np.str_('virginica')]
  
   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target target_name  
0       0      setosa  
1       0      setosa  
2       0      setosa  
3       0      setosa  
4       0      setosa  


In [11]:
# Save dataset locally
data_dir = Path('../data')
data_dir.mkdir(exist_ok=True)

# Save as CSV
csv_path = data_dir / 'iris_dataset.csv'
df.to_csv(csv_path, index=False)
print(f"✅ Saved CSV: {csv_path}")

# Save as NumPy arrays
npz_path = data_dir / 'iris_dataset.npz'
np.savez(npz_path, X=X, y=y, feature_names=iris.feature_names, target_names=iris.target_names)
print(f"✅ Saved NumPy: {npz_path}")

# Save metadata
metadata = {
    'name': 'iris',
    'description': 'Iris flower classification dataset',
    'n_samples': len(X),
    'n_features': X.shape[1],
    'n_classes': len(iris.target_names),
    'feature_names': iris.feature_names,
    'target_names': iris.target_names.tolist()
}

metadata_path = data_dir / 'iris_metadata.pkl'
with open(metadata_path, 'wb') as f:
    pickle.dump(metadata, f)
print(f"✅ Saved metadata: {metadata_path}")

✅ Saved CSV: ../data/iris_dataset.csv
✅ Saved NumPy: ../data/iris_dataset.npz
✅ Saved metadata: ../data/iris_metadata.pkl


In [12]:
# Upload dataset to Cloud Storage
print(f"☁️ Uploading dataset to GCS...")

# Upload CSV file
blob = bucket.blob('data/iris_dataset.csv')
blob.upload_from_filename(str(csv_path))
print(f"✅ Uploaded: gs://{BUCKET_NAME}/data/iris_dataset.csv")

# Upload NumPy file
blob = bucket.blob('data/iris_dataset.npz')
blob.upload_from_filename(str(npz_path))
print(f"✅ Uploaded: gs://{BUCKET_NAME}/data/iris_dataset.npz")

# Upload metadata
blob = bucket.blob('data/iris_metadata.pkl')
blob.upload_from_filename(str(metadata_path))
print(f"✅ Uploaded: gs://{BUCKET_NAME}/data/iris_metadata.pkl")

print(f"\n🎉 Dataset preparation complete!")

☁️ Uploading dataset to GCS...
✅ Uploaded: gs://mlops-vertex-ai-bucket-295610/data/iris_dataset.csv
✅ Uploaded: gs://mlops-vertex-ai-bucket-295610/data/iris_dataset.npz
✅ Uploaded: gs://mlops-vertex-ai-bucket-295610/data/iris_metadata.pkl

🎉 Dataset preparation complete!


## ✅ Section 7: Environment Summary

Final validation of our complete environment setup.

In [13]:
# Import our project utilities
import sys
sys.path.append('../src')

from config import config
from utils import validate_gcp_environment

print("🔍 Final Environment Validation")
print("=" * 50)

# Validate configuration
print(f"\n📋 Configuration:")
print(f"  Project ID: {config.project_id}")
print(f"  Region: {config.region}")
print(f"  Bucket: {config.bucket_name}")
print(f"  Model Name: {config.model_name}")

# Validate GCP environment
validation = validate_gcp_environment()
print(f"\n🔧 GCP Environment:")
for component, status in validation.items():
    status_icon = "✅" if status else "❌"
    print(f"  {status_icon} {component.replace('_', ' ').title()}: {status}")

# Check datasets
data_files = [
    '../data/iris_dataset.csv',
    '../data/iris_dataset.npz', 
    '../data/iris_metadata.pkl'
]

print(f"\n📊 Local Datasets:")
for file_path in data_files:
    exists = Path(file_path).exists()
    status_icon = "✅" if exists else "❌"
    print(f"  {status_icon} {Path(file_path).name}: {exists}")

# Check GCS bucket contents
print(f"\n☁️ Cloud Storage:")
try:
    blobs = list(bucket.list_blobs(prefix='data/'))
    for blob in blobs:
        if not blob.name.endswith('.gitkeep'):
            print(f"  ✅ gs://{BUCKET_NAME}/{blob.name} ({blob.size} bytes)")
except Exception as e:
    print(f"  ❌ Error accessing bucket: {e}")

print(f"\n🎉 Phase 1 Complete! Environment is ready for Phase 2.")
print(f"\nNext step: Create data processing pipeline notebook")

🔍 Final Environment Validation

📋 Configuration:
  Project ID: mlops-295610
  Region: us-central1
  Bucket: mlops-vertex-ai-bucket-295610
  Model Name: iris-classifier

🔧 GCP Environment:
  ✅ Gcloud Cli: True
  ❌ Application Credentials: False
  ✅ Project Id: True

📊 Local Datasets:
  ✅ iris_dataset.csv: True
  ✅ iris_dataset.npz: True
  ✅ iris_metadata.pkl: True

☁️ Cloud Storage:
  ✅ gs://mlops-vertex-ai-bucket-295610/data/iris_dataset.csv (4189 bytes)
  ✅ gs://mlops-vertex-ai-bucket-295610/data/iris_dataset.npz (7396 bytes)
  ✅ gs://mlops-vertex-ai-bucket-295610/data/iris_metadata.pkl (275 bytes)

🎉 Phase 1 Complete! Environment is ready for Phase 2.

Next step: Create data processing pipeline notebook


---

## 🎯 Summary

You have successfully completed **Phase 1: Environment Setup & Foundation**!

### What we accomplished:

✅ **Python Environment** - Validated all required packages  
✅ **Google Cloud Authentication** - Set up project credentials  
✅ **API Access** - Verified Vertex AI and Cloud Storage APIs  
✅ **Cloud Storage** - Created bucket with proper structure  
✅ **Dataset Preparation** - Downloaded and uploaded Iris dataset  
✅ **Project Configuration** - Set up config files and utilities  

### Next Steps:

Now you're ready to move to **Phase 2: Data Pipeline Implementation**

- Create notebook: `02_data_processing_pipeline.ipynb`
- Implement data preprocessing and validation
- Set up train/test splits
- Create data processing components

---

**🔗 Useful Links:**
- [Google Cloud Console](https://console.cloud.google.com)
- [Vertex AI Console](https://console.cloud.google.com/vertex-ai)
- [Cloud Storage Browser](https://console.cloud.google.com/storage/browser)
- [Project Documentation](../README.md)